In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import lyricsgenius
import selenium
import sys
import spotipy
import spotipy.util as util

#Import all the relevant libraries we'll be using for Selenium
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common import keys
from selenium.webdriver.common.action_chains import ActionChains


In [7]:
#To prevent interruptions with selenium, we'll be configuring a number of options
chrome_options = webdriver.ChromeOptions()
chrome_options.accept_untrusted_certs = True
chrome_options.assume_untrusted_cert_issuer = True
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-impl-side-painting")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--disable-seccomp-filter-sandbox")
chrome_options.add_argument("--disable-breakpad")
chrome_options.add_argument("--disable-client-side-phishing-detection")
chrome_options.add_argument("--disable-cast")
chrome_options.add_argument("--disable-cast-streaming-hw-encoding")
chrome_options.add_argument("--disable-cloud-import")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--disable-session-crashed-bubble")
chrome_options.add_argument("--disable-ipv6")
chrome_options.add_argument("--allow-http-screen-capture")
chrome_options.add_argument("--start-maximized")

In [8]:
#open up chrome browser and navigate to genius
opts = Options()
opts.set_headless()
assert opts.headless  
browser = webdriver.Chrome()


C:\Users\Owen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use setter for headless property instead of set_headless
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# pd.DataFrame(rapper_list_for_cross[:465]).to_csv('rapper_names_searcher_1.csv', header=None, index=False)

In [10]:
# pd.DataFrame(rapper_list_for_cross[465:929]).to_csv('rapper_names_searcher_2.csv', header=None, index=False)

In [11]:
# pd.DataFrame(rapper_list_for_cross[929:]).to_csv('rapper_names_searcher_3.csv', header=None, index=False)

In [12]:
# #Pull our list in
# rapper_list_for_cross = list(pd.read_csv('rapper_names_searcher_1.csv', header=None)[0])

In [13]:
#We will start by initiating empty lists to which we'll add our information on each pull (artist name, # follows, follower names)
genius_search_results_artist_names = []
genius_search_results_artist_follows = []
genius_search_results_artist_follower_names = []

#For timing and for indexing our lists for printing, we have some variables we will initiate here
program_starts = time.time()  
iteration_check = 0

#Iterate through our list of rappers
for artist in ['chance the rapper', '50 cent', 'lil baby']:
    
    #Open our headless chrome browser
    browser.get('https://genius.com/search?q=' + artist)
    user_names =[]        
    
    try:
        
        #Wait until the artist section of the search results is detected
        WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, "//html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[2]/div[4]/search-result-section/div/div[2]/search-result-items/div[1]"))
        ) 
        
        #Click on the top artist result
        top_artist_result = browser.find_element_by_xpath("//html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[2]/div[4]/search-result-section/div/div[2]/search-result-items/div[1]")     
        top_artist_result.click()
        
        
        try:
            #Wait until the "Followers" element is located
            artist_name = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//html/body/ul[1]/li[3]/a/span'))
            )
            
            #Append our artist name to our list 
            genius_search_results_artist_names.append(artist_name.text)
            
        except:
            print('error @ first except')
            #Skip this artist if we get an error because nothing appears
            genius_search_results_artist_names.append('')
                
        try:
            
            #Once we're on the artist page, wait 
            artist_followers = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//html/body/routable-page/ng-outlet/routable-profile-page/ng-outlet/routed-page/profile-page/div[2]/div[2]/div[2]/div/h3[2]/span'))
            )
           
            #Grab the number of followers
            genius_search_results_artist_follows.append(artist_followers.text)
            
            #Wait and click on followers
            time.sleep(1)
            artist_followers.click()
            
            try:
                
                #Scroll down after we've clicked on followers to populate list of followers
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(2)
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(2)

                #Find all users by their badges
                follower_names = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'user_badge-login')) )

                #iterate through all the users we found and build list of followers. Append this to larger list

                user_names_response = browser.find_elements_by_class_name('user_badge-login')
                for user in user_names_response:
                    user_names.append(user.text)
                genius_search_results_artist_follower_names.append(user_names)

            except:
                #If we encounter any issues with retrieving follower names, just skip it
                user_names.append('')
                genius_search_results_artist_follower_names.append(user_names)
                print('error @ second except')
            
            
        except:
            genius_search_results_artist_names.append('')
            genius_search_results_artist_follows.append('')
            print(f'ERROR @ fourth except for {artist}.')            
            user_names.append('')
            genius_search_results_artist_follower_names.append(user_names)
            
    except:
            genius_search_results_artist_names.append('')
            genius_search_results_artist_follows.append('')
            print(f'ERROR @ fourth except for {artist}.')            
            user_names.append('')
            genius_search_results_artist_follower_names.append(user_names)
   
    #Check how long it took. Print our current status so we can keep track    
    now = time.time()
    diff = now - program_starts
    print(f"It has been {round(diff,2)} seconds since we started.")
    iteration_check += 1  



It has been 11.91 seconds since we started.
It has been 21.25 seconds since we started.
It has been 31.53 seconds since we started.


In [26]:
rappers_1_df = pd.DataFrame(zip(rapper_list_for_cross,genius_search_results_artist_names, genius_search_results_artist_follows,genius_search_results_artist_follower_names))
rappers_1_df.to_csv('combined_rapper_data_1.csv', index=False)